In [1]:
import pandas as pd

In [29]:
entree = ['01 ', '011', '031', '041', '043', '051', '1', '11','11 ','17', '17 ', '31', '41', '43', '51' ]
sortie = ['02 ', '052', '2', '52']
entree_c = ['03 ', '056', '22', '22 ', '3', '42', '56']
sortie_c = ['04 ', '057', '4', '57'] 
columns = ['client_id','date',  'O', 'D', 'Ot', 'Dt','mode_TRNS', 'mode_RES' ,
           'O_station','D_station','ligne', 'unkonwn','O_zones','O_x','O_y','D_zones','D_x','D_y', 'O_mode', 'D_mode']

In [3]:
def get_counts(anything):
    counts = {}
    for a in anything:
        if a in counts:
            counts[a] = counts[a] + 1
        else:
            counts[a] = 1
    return counts


In [32]:
def  trip_prediction(data, n, index):
    i = 0
    transfer_id = 0
    trips = pd.DataFrame([], columns=columns)
    start = data.loc[index]     #start would be used in the final trip
    end = data.loc[index+n-1]
    while i+1 < n:
        first = data.loc[index+i]
        second = data.loc[index+i+1]
        first_location = arrets[(arrets['COD_TLB_ARRT_STTN'] == first['COD_TLB_ARRT_STTN'])
                                &(arrets['LIBL_LONG_ARRT_STTN'] == first['Lib_Arret'])]
        second_location = arrets[(arrets['COD_TLB_ARRT_STTN'] == second['COD_TLB_ARRT_STTN'])
                                &(arrets['LIBL_LONG_ARRT_STTN'] == second['Lib_Arret'])]
        if (len(first_location) == 0) or (len(second_location) == 0):
            return pd.DataFrame()
        first_zones, first_location_x, first_location_y = first_location.iloc[0][['ZONS_ARRT',
                                                                                  'COOR_X_PHSQ',
                                                                                  'COOR_Y_PHSQ']].get_values()
        second_zones, second_location_x, second_location_y = second_location.iloc[0][['ZONS_ARRT',
                                                                                      'COOR_X_PHSQ', 
                                                                                      'COOR_Y_PHSQ']].get_values()       
        first_data = [first['NUMR_CART'], first['JOUR_EXPL_VALD'], 
                      first['Lib_Arret'], second['Lib_Arret'], first['HEUR_VALD'], second['HEUR_VALD'], first['COD_STF_TRNS'],
                      first['COD_STF_RES'],first['COD_TLB_ARRT_STTN'],second['COD_TLB_ARRT_STTN'],first['COD_STF_LIGN'], 'd', first_zones,
                      first_location_x, first_location_y, second_zones, 
                      second_location_x, second_location_y, first['COD_MOD_VALD'], second['COD_MOD_VALD']]
        trips.loc[transfer_id] = first_data
        if second['COD_NATR_VALD'] in sortie:
            trips.loc[transfer_id, 'unkonwn'] = 'no'
            transfer_id = transfer_id + 1 
            i = i + 2
        else:
            transfer_id = transfer_id + 1
            i = i + 1
    if (i+1) == n:  # which means the last trip is a 'E'   
        start_location =  arrets[(arrets['COD_TLB_ARRT_STTN'] == start['COD_TLB_ARRT_STTN'])
                                 &(arrets['LIBL_LONG_ARRT_STTN'] == start['Lib_Arret'])]
        end_location = arrets[(arrets['COD_TLB_ARRT_STTN'] == end['COD_TLB_ARRT_STTN'])
                              &(arrets['LIBL_LONG_ARRT_STTN'] == end['Lib_Arret'])]
        if (len(start_location)==0) or (len(end_location)==0):
            return pd.DataFrame()
        start_zones, start_location_x, start_location_y = start_location.iloc[0][['ZONS_ARRT',
                                                                                  'COOR_X_PHSQ',
                                                                                  'COOR_Y_PHSQ']].get_values()
        
        end_zones, end_location_x, end_location_y = end_location.iloc[0][['ZONS_ARRT',
                                                                          'COOR_X_PHSQ',
                                                                          'COOR_Y_PHSQ']].get_values()
        last_data = [end['NUMR_CART'], end['JOUR_EXPL_VALD'], 
                     end['Lib_Arret'], start['Lib_Arret'], end['HEUR_VALD'], '', end['COD_STF_TRNS'], 
                     end['COD_STF_RES'], end['COD_TLB_ARRT_STTN'],start['COD_TLB_ARRT_STTN'], 
                     end['COD_STF_LIGN'], 'dh', end_zones,end_location_x,end_location_y, 
                    start_zones,start_location_x,start_location_y, end['COD_MOD_VALD'], start['COD_MOD_VALD']]
        trips.loc[transfer_id] = last_data
    return trips

In [14]:
def  output(data, n):
    trips = pd.DataFrame()
    for index in range(int(len(data)/n)):
        a= trip_prediction(data, n, index*n)
        trips =  trips.append(a, ignore_index = True)
    return trips


In [6]:
path_3 =  'F:/Stage/data/AFC/counts/counts_3'
path_1 = 'F:/Stage/data/AFC/counts/counts_1'
path_arret = "F:/Stage/data/AFC/DM arret.txt"
path_2 = 'F:/Stage/data/AFC/counts/counts_2'
path_4 =  'F:/Stage/data/AFC/counts/counts_4'
path_5 =  'F:/Stage/data/AFC/counts/counts_5'
path_6 =  'F:/Stage/data/AFC/counts/counts_6'
path_7 =  'F:/Stage/data/AFC/counts/counts_7'

In [7]:
data_3 = pd.read_csv(path_3, delimiter="\t", header = 0, encoding="latin-1")
data_3['COD_NATR_VALD'] = data_3['COD_NATR_VALD'].astype('str')
data_3['HEUR_VALD'] = pd.to_timedelta(data_3['HEUR_VALD'])
data_3_sort = data_3.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_3_sort = data_3_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data_4 = pd.read_csv(path_4, delimiter="\t", header = 0, encoding="latin-1")
data_4['COD_NATR_VALD'] = data_4['COD_NATR_VALD'].astype('str')
data_4['HEUR_VALD'] = pd.to_timedelta(data_4['HEUR_VALD'])
data_4_sort = data_4.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_4_sort = data_4_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data_5 = pd.read_csv(path_5, delimiter="\t", header = 0, encoding="latin-1")
data_5['COD_NATR_VALD'] = data_5['COD_NATR_VALD'].astype('str')
data_5['HEUR_VALD'] = pd.to_timedelta(data_5['HEUR_VALD'])
data_5_sort = data_5.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_5_sort = data_5_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data_7 = pd.read_csv(path_7, delimiter="\t", header = 0, encoding="latin-1")
data_7['COD_NATR_VALD'] = data_7['COD_NATR_VALD'].astype('str')
data_7['HEUR_VALD'] = pd.to_timedelta(data_7['HEUR_VALD'])
data_7_sort = data_7.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_7_sort = data_7_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_6 = pd.read_csv(path_6, delimiter="\t", header = 0, encoding="latin-1")
data_6['COD_NATR_VALD'] = data_6['COD_NATR_VALD'].astype('str')
data_6['HEUR_VALD'] = pd.to_timedelta(data_6['HEUR_VALD'])
data_6_sort = data_6.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_6_sort = data_6_sort.reset_index(drop = True)

In [4]:
data_2 = pd.read_csv(path_2, delimiter="\t", header = 0, encoding="latin-1")
data_2['COD_NATR_VALD'] = data_2['COD_NATR_VALD'].astype('str')
data_2['HEUR_VALD'] = pd.to_timedelta(data_2['HEUR_VALD'])
data_2_sort = data_2.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_2_sort = data_2_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data_2_sort[data_2_sort['NUMR_CART'] == '99988615879853555405']

,Unnamed: 0,NUMR_CART,JOUR_EXPL_VALD,HEUR_VALD,Lib_Arret,COD_TLB_TRNS,COD_TLB_RES,COD_TLB_ARRT_STTN,NUMR_BORN_VALD_EQPM,COD_NATR_VALD,COD_MOD_VALD,COD_STF_TRNS,COD_STF_RES,COD_STF_LIGN
168,12475602,99988615879853555405,14/03/2017,06:10:33,SAINT-DENIS,2,002,271015,20500505,1,1,800,853,ND
169,12475690,99988615879853555405,14/03/2017,16:11:09,SAINT-DENIS,2,002,271015,14500505,2,1,800,853,ND


In [6]:
data_2_sort[data_2_sort['NUMR_CART'] == '99985864111412513744']

,Unnamed: 0,NUMR_CART,JOUR_EXPL_VALD,HEUR_VALD,Lib_Arret,COD_TLB_TRNS,COD_TLB_RES,COD_TLB_ARRT_STTN,NUMR_BORN_VALD_EQPM,COD_NATR_VALD,COD_MOD_VALD,COD_STF_TRNS,COD_STF_RES,COD_STF_LIGN
222,10046219,99985864111412513744,14/03/2017,08:23:01,LA DEFENSE-GRANDE ARCHE,1,801,0030050170401,114,52,1,810,801,ND
223,10046131,99985864111412513744,14/03/2017,14:51:21,LA DEFENSE-GRANDE ARCHE,1,801,0030050170401,201,51,1,810,801,ND


In [9]:
data_2_sort[data_2_sort['NUMR_CART'] == '99965752444344107009']

,Unnamed: 0,NUMR_CART,JOUR_EXPL_VALD,HEUR_VALD,Lib_Arret,COD_TLB_TRNS,COD_TLB_RES,COD_TLB_ARRT_STTN,NUMR_BORN_VALD_EQPM,COD_NATR_VALD,COD_MOD_VALD,COD_STF_TRNS,COD_STF_RES,COD_STF_LIGN
512,75065,99965752444344107009,14/03/2017,07:55:13,POISSY,1,801,386573,12321002,1,1,810,801,ND
513,75153,99965752444344107009,14/03/2017,18:33:19,POISSY,1,801,386573,15321002,2,1,810,801,ND


In [25]:
get_counts(data_3_sort['COD_MOD_VALD'])

{-1: 6, 1: 404013, 2: 393623, 3: 540069, 4: 53865, 6: 1309}

In [10]:
arrets = pd.read_csv(path_arret, delimiter="\t", header= 0, encoding="latin-1")
arrets = arrets[arrets.ZONS_ARRT != 'ND ']
arrets = arrets[arrets.COOR_X_PHSQ != '-1']
arrets = arrets[arrets.COOR_X_PHSQ != '?']
arrets['COOR_X_PHSQ'] = arrets['COOR_X_PHSQ'].apply(lambda x: float((x.split(',')[0])))
arrets['COOR_Y_PHSQ'] = arrets['COOR_Y_PHSQ'].apply(lambda x: float((x.split(',')[0])))

In [8]:
select_data = data_3_sort.iloc[:100]
select_data


,Unnamed: 0,NUMR_CART,JOUR_EXPL_VALD,HEUR_VALD,Lib_Arret,COD_TLB_TRNS,COD_TLB_RES,COD_TLB_ARRT_STTN,NUMR_BORN_VALD_EQPM,COD_NATR_VALD,COD_MOD_VALD,COD_STF_TRNS,COD_STF_RES,COD_STF_LIGN
0,6289284,99999942845366768893,14/03/2017,10:57:31,PYRENEES,3,001,0030030241400,101,31,3,100,110,ND
1,6289196,99999942845366768893,14/03/2017,17:30:51,SAINT-GERMAIN-DES-PRES,3,001,0030030021500,232,31,3,100,110,ND
2,1723316,99999942845366768893,14/03/2017,18:50:29,CHATELET,3,001,0030030010800,401,31,3,100,110,ND
3,7153827,99999577198461515588,14/03/2017,08:00:58,HOTEL DE VILLE DE FONTENAY,3,001,0030010124016,2125,11,2,100,100,124
4,7153915,99999577198461515588,14/03/2017,08:20:46,VAL DE FONTENAY RER,3,001,0030010118015,6803,11,2,100,100,118
5,11376859,99999577198461515588,14/03/2017,19:25:28,VAL DE FONTENAY RER,3,001,0030010124022,9042,11,2,100,100,124
6,354119,99999492695837542235,14/03/2017,08:02:30,ESCADRILLE NORMANDIE-NIEMEN - PAUL VAILLANT-CO...,3,001,0030010151012,611,11,2,100,100,151
7,354031,99999492695837542235,14/03/2017,10:11:44,PLACE DU 19 MARS 1962,3,001,0030010151034,11948,11,2,100,100,151
8,354207,99999492695837542235,14/03/2017,12:29:00,ESCADRILLE NORMANDIE-NIEMEN - PAUL VAILLANT-CO...,3,001,0030010151012,1753,11,2,100,100,151
9,6550859,99998853895565292735,14/03/2017,07:56:20,MAIRIE DE ROMAINVILLE,3,001,0030010105047,983,11,2,100,100,105


18268.0

In [33]:
output_file = output(select_data, 3)

In [34]:
output_file

,client_id,date,O,D,Ot,Dt,mode_TRNS,mode_RES,O_station,D_station,ligne,unkonwn,O_zones,O_x,O_y,D_zones,D_x,D_y,O_mode,D_mode
0,99999942845366768893,14/03/2017,PYRENEES,SAINT-GERMAIN-DES-PRES,10:57:31,0 days 17:30:51,100,110,0030030241400,0030030021500,ND,d,1,603578.0,130382.0,1,599798.0,128139.0,3,3
1,99999942845366768893,14/03/2017,SAINT-GERMAIN-DES-PRES,CHATELET,17:30:51,0 days 18:50:29,100,110,0030030021500,0030030010800,ND,d,1,599798.0,128139.0,1,600864.0,128566.0,3,3
2,99999942845366768893,14/03/2017,CHATELET,PYRENEES,18:50:29,,100,110,0030030010800,0030030241400,ND,dh,1,600864.0,128566.0,1,603578.0,130382.0,3,3
3,99999577198461515588,14/03/2017,HOTEL DE VILLE DE FONTENAY,VAL DE FONTENAY RER,08:00:58,0 days 08:20:46,100,100,0030010124016,0030010118015,124,d,3,610411.0,127523.0,3,611092.0,128164.0,2,2
4,99999577198461515588,14/03/2017,VAL DE FONTENAY RER,VAL DE FONTENAY RER,08:20:46,0 days 19:25:28,100,100,0030010118015,0030010124022,118,d,3,611092.0,128164.0,3,611078.0,128101.0,2,2
5,99999577198461515588,14/03/2017,VAL DE FONTENAY RER,HOTEL DE VILLE DE FONTENAY,19:25:28,,100,100,0030010124022,0030010124016,124,dh,3,611078.0,128101.0,3,610411.0,127523.0,2,2
6,99999492695837542235,14/03/2017,ESCADRILLE NORMANDIE-NIEMEN - PAUL VAILLANT-CO...,PLACE DU 19 MARS 1962,08:02:30,0 days 10:11:44,100,100,0030010151012,0030010151034,151,d,3,607422.0,134759.0,3,608448.0,135511.0,2,2
7,99999492695837542235,14/03/2017,PLACE DU 19 MARS 1962,ESCADRILLE NORMANDIE-NIEMEN - PAUL VAILLANT-CO...,10:11:44,0 days 12:29:00,100,100,0030010151034,0030010151012,151,d,3,608448.0,135511.0,3,607422.0,134759.0,2,2
8,99999492695837542235,14/03/2017,ESCADRILLE NORMANDIE-NIEMEN - PAUL VAILLANT-CO...,ESCADRILLE NORMANDIE-NIEMEN - PAUL VAILLANT-CO...,12:29:00,,100,100,0030010151012,0030010151012,151,dh,3,607422.0,134759.0,3,607422.0,134759.0,2,2
9,99998853895565292735,14/03/2017,MAIRIE DE ROMAINVILLE,MAIRIE DES LILAS,07:56:20,0 days 08:10:03,100,100,0030010105047,0030030230300,105,d,3,607221.0,131628.0,2,605864.0,131038.0,2,3


In [13]:
output_path = 'E:/Study/data/7_0.csv'
output_file.to_csv(output_path, header=True,sep="\t", encoding='latin-1')